In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 카카오 맵 API 불러오기 위해
import requests
import json

# 상관분석
from sklearn import preprocessing
from scipy.stats import chi2_contingency

## 데이터셋 불러오기

In [11]:
mapo_c = pd.read_csv('data/서울특별시 마포구_흡연시설 현황_20220418.csv')
print(mapo_c.shape)
mapo_c.head()

(357, 9)


,시도명,시군구명,상호명,소재지도로명주소,업종,흡연실구분,흡연시설형태,데이터기준일자,Unnamed: 8
0,서울특별시,마포구,3POP 빈티지 PC CAFE,서울특별시 마포구 월드컵북로30길 13,게임제공업소,실내,밀폐형,44669.0,NaN
1,서울특별시,마포구,이룸PC,서울특별시 마포구 마포대로 38,게임제공업소,실내,밀폐형,44669.0,NaN
2,서울특별시,마포구,제노PC방(홍대2),서울특별시 마포구 와우산로 83,게임제공업소,실내,밀폐형,44669.0,NaN
3,서울특별시,마포구,I PC CAFE 홍대역점,서울특별시 마포구 양화로 156,게임제공업소,실내,밀폐형,44669.0,NaN
4,서울특별시,마포구,히어로PC방,서울특별시 마포구 독막로 241,게임제공업소,실내,밀폐형,44669.0,NaN


In [15]:
mapo_cbb = pd.read_csv('data/쓰레기통_위경도/마포구_담배꽁초쓰레기통_위경도.csv')
print(mapo_cbb.shape)
mapo_cbb.head()

(65, 8)


,시군구명,address_name,경도,위도,도로명주소,상호명,수량,세부위치
0,마포구,서울 마포구 광성로6길 66,126.940168,37.547265,서울특별시 마포구 광성로6길 66,마포나루장터,1,NaN
1,마포구,서울 마포구 대흥로 40,126.938931,37.544230,서울특별시 마포구 대흥로 40,블랙시스(아이나비),1,가게앞
2,마포구,서울 마포구 마포대로1길 26,126.942323,37.540314,서울특별시 마포구 마포대로1길 26,성우서서갈비,2,가게안/가게앞
3,마포구,서울 마포구 마포대로1길 46,126.940477,37.541640,서울특별시 마포구 마포대로1길 46,고고네 닭한마리,1,가게앞
4,마포구,서울 마포구 마포대로1길 46,126.940477,37.541640,서울특별시 마포구 마포대로1길 46 우측상가,원마산아구찜,1,재활용수집


In [16]:
mapo_tugi = pd.read_csv('data/서울특별시 마포구 담배꽁초 상습 무단투기 집중관리 지역_20220805.csv')
print(mapo_tugi.shape)
mapo_tugi.head()

(4, 6)


,시도명,시군구명,도로명주소,세부위치,데이터기준일자,Unnamed: 5
0,서울특별시,마포구,서울특별시 마포구 양화로 156,홍대입구역 9번출구 LG팰리스 뒤쪽 도로,2022-08-05,NaN
1,서울특별시,마포구,서울특별시 마포구 양화로 183,홍대입구역 3번출구 인근,2022-08-05,NaN
2,서울특별시,마포구,서울특별시 마포구 마포대로 144,마포T타운 뒤쪽 도로,2022-08-05,NaN
3,서울특별시,마포구,서울특별시 마포구 상암산로 38,kbs 건물 인근,2022-08-05,NaN


In [12]:
mapo_c = mapo_c[['소재지도로명주소']]

In [17]:
mapo_cbb = mapo_cbb[['도로명주소']]

In [18]:
mapo_tugi = mapo_tugi[['도로명주소']]

### mapo_c(흡연구역) 대로명 뽑기

In [20]:
a = mapo_c['소재지도로명주소'].apply(lambda x:x.split(" "))
a

0          [서울특별시, 마포구, 월드컵북로30길, 13]
1              [서울특별시, 마포구, 마포대로, 38]
2              [서울특별시, 마포구, 와우산로, 83]
3              [서울특별시, 마포구, 양화로, 156]
4              [서울특별시, 마포구, 독막로, 241]
                    ...              
352      [서울특별시, 마포구, 독막로, 17, (합정동)]
353         [서울특별시, 마포구, 양화로18안길, 12]
354    [서울특별시, 마포구, 새창로2길, 13, (도화동)]
355         [서울특별시, 마포구, 월드컵로42길, 13]
356    [서울특별시, 마포구, 월드컵로, 212, (성산동)]
Name: 소재지도로명주소, Length: 357, dtype: object

In [21]:
for i in range(len(a)):
    a[i] = a[i][2]

In [22]:
mapo_c['소재지도로명주소']=a

In [23]:
b = mapo_c['소재지도로명주소'].str.split('로')
b

0      [월드컵북, 30길]
1          [마포대, ]
2          [와우산, ]
3           [양화, ]
4           [독막, ]
          ...     
352         [독막, ]
353     [양화, 18안길]
354       [새창, 2길]
355     [월드컵, 42길]
356        [월드컵, ]
Name: 소재지도로명주소, Length: 357, dtype: object

In [24]:
for i in range(len(b)):
    b[i] = b[i][0] + '로'

In [25]:
mapo_c['소재지도로명주소'] = b

### mapo_cbb(쓰레기통) 대로명 뽑기

In [26]:
a = mapo_cbb['도로명주소'].apply(lambda x:x.split(" "))

for i in range(len(a)):
    a[i] = a[i][2]

mapo_cbb['도로명주소']=a

b = mapo_cbb['도로명주소'].str.split('로')

for i in range(len(b)):
    b[i] = b[i][0] + '로'

mapo_cbb['도로명주소'] = b

### mapo_tugi(투기지역) 대로명 뽑기

In [27]:
a = mapo_tugi['도로명주소'].apply(lambda x:x.split(" "))

for i in range(len(a)):
    a[i] = a[i][2]

mapo_tugi['도로명주소']=a

b = mapo_tugi['도로명주소'].str.split('로')

for i in range(len(b)):
    b[i] = b[i][0] + '로'

mapo_tugi['도로명주소'] = b

### concat

In [28]:
mapo_m_1 = pd.concat([mapo_c, mapo_cbb], axis=1)
mapo_m_1

,소재지도로명주소,도로명주소
0,월드컵북로,광성로
1,마포대로,대흥로
2,와우산로,마포대로
3,양화로,마포대로
4,독막로,마포대로
...,...,...
352,독막로,NaN
353,양화로,NaN
354,새창로,NaN
355,월드컵로,NaN


In [35]:
mapo_m = pd.concat([mapo_m_1, mapo_tugi], axis=1)
mapo_m

,소재지도로명주소,도로명주소,도로명주소
0,월드컵북로,광성로,양화로
1,마포대로,대흥로,양화로
2,와우산로,마포대로,마포대로
3,양화로,마포대로,상암산로
4,독막로,마포대로,NaN
...,...,...,...
352,독막로,NaN,NaN
353,양화로,NaN,NaN
354,새창로,NaN,NaN
355,월드컵로,NaN,NaN


In [36]:
mapo_m.columns = ['흡연구역', '쓰레기통', '투기지역']

## 검정

In [37]:
label = preprocessing.LabelEncoder()
data_encoded = pd.DataFrame()

for i in mapo_m.columns:
    data_encoded[i] = label.fit_transform(mapo_m[i])
    
data_encoded.head()

,흡연구역,쓰레기통,투기지역
0,31,0,2
1,7,1,2
2,29,2,0
3,26,2,1
4,4,2,3


### 쓰레기통, 흡연구역 상관

In [45]:
df_crosstab = pd.crosstab(data_encoded['쓰레기통'], data_encoded['흡연구역'])
df_crosstab

흡연구역,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
쓰레기통,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0,0,1,2,4,1,0,5,0,1,...,7,9,7,0,0,0,4,0,0,0
4,1,3,4,3,11,14,3,42,6,5,...,9,13,47,8,2,1,19,6,1,2


In [46]:
rows = [row.to_list() for i, row in df_crosstab.iterrows()]
rows

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  2,
  4,
  1,
  0,
  5,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  2,
  1,
  0,
  2,
  0,
  0,
  1,
  4,
  3,
  2,
  7,
  9,
  7,
  0,
  0,
  0,
  4,
  0,
  0,
  0],
 [1,
  3,
  4,
  3,
  11,
  14,
  3,
  42,
  6,
  5,
  9,
  5,
  2,
  3,
  12,
  6,
  1,
  3,
  3,
  2,
  2,
  2,
  2,
  10,
  1,
  8,
  18,
  2,
  1,
  9,
  13,
  47,
  8,
  2,
  1,
  19,
  6,
  1,
  2]]

In [44]:
# 쓰레기통, 흡연구역 상관 없음
chi2_contingency(rows, correction=False)

(83.8033068180671,
 0.9999985859456437,
 152,
 array([[2.80112045e-03, 8.40336134e-03, 1.40056022e-02, 1.40056022e-02,
         4.48179272e-02, 4.20168067e-02, 8.40336134e-03, 1.34453782e-01,
         1.68067227e-02, 1.68067227e-02, 2.80112045e-02, 1.40056022e-02,
         5.60224090e-03, 1.12044818e-02, 3.64145658e-02, 1.68067227e-02,
         2.80112045e-03, 8.40336134e-03, 8.40336134e-03, 1.12044818e-02,
         8.40336134e-03, 5.60224090e-03, 1.12044818e-02, 2.80112045e-02,
         2.80112045e-03, 2.52100840e-02, 6.44257703e-02, 1.40056022e-02,
         8.40336134e-03, 4.76190476e-02, 6.44257703e-02, 1.54061625e-01,
         2.24089636e-02, 5.60224090e-03, 2.80112045e-03, 6.44257703e-02,
         1.68067227e-02, 2.80112045e-03, 5.60224090e-03],
        [2.80112045e-03, 8.40336134e-03, 1.40056022e-02, 1.40056022e-02,
         4.48179272e-02, 4.20168067e-02, 8.40336134e-03, 1.34453782e-01,
         1.68067227e-02, 1.68067227e-02, 2.80112045e-02, 1.40056022e-02,
         5.60224090e

In [47]:
x2 = chi2_contingency(rows, correction=False)[0]
n = np.sum(rows)
minDimension = min(np.array(rows).shape)-1

V = np.sqrt((x2/n) / minDimension)

print(V)

0.24225149966157541


### 쓰레기통, 투기지역 상관

In [48]:
df_crosstab = pd.crosstab(data_encoded['쓰레기통'], data_encoded['투기지역'])
df_crosstab

투기지역,0,1,2,3
쓰레기통,,,,
0,0,0,1,0
1,0,0,1,0
2,1,1,0,2
3,0,0,0,59
4,0,0,0,292


In [49]:
rows = [row.to_list() for i, row in df_crosstab.iterrows()]
rows

[[0, 0, 1, 0], [0, 0, 1, 0], [1, 1, 0, 2], [0, 0, 0, 59], [0, 0, 0, 292]]

In [50]:
# 쓰레기통, 투기지역 상관 있음
chi2_contingency(rows, correction=False)

(534.4886685552408,
 1.0018268155530567e-106,
 12,
 array([[2.80112045e-03, 2.80112045e-03, 5.60224090e-03, 9.88795518e-01],
        [2.80112045e-03, 2.80112045e-03, 5.60224090e-03, 9.88795518e-01],
        [1.12044818e-02, 1.12044818e-02, 2.24089636e-02, 3.95518207e+00],
        [1.65266106e-01, 1.65266106e-01, 3.30532213e-01, 5.83389356e+01],
        [8.17927171e-01, 8.17927171e-01, 1.63585434e+00, 2.88728291e+02]]))

In [51]:
# 쓰레기통, 투기지역 상관 있음
x2 = chi2_contingency(rows, correction=False)[0]
n = np.sum(rows)
minDimension = min(np.array(rows).shape)-1

V = np.sqrt((x2/n) / minDimension)

print(V)

0.70643875384688


### 흡연구역, 투기지역 상관

In [58]:
df_crosstab = pd.crosstab(data_encoded['흡연구역'], data_encoded['투기지역'])
df_crosstab

투기지역,0,1,2,3
흡연구역,,,,
0,0,0,0,1
1,0,0,0,3
2,0,0,0,5
3,0,0,0,5
4,0,0,0,16
5,0,0,0,15
6,0,0,0,3
7,0,0,1,47
8,0,0,0,6


In [59]:
rows = [row.to_list() for i, row in df_crosstab.iterrows()]
rows

[[0, 0, 0, 1],
 [0, 0, 0, 3],
 [0, 0, 0, 5],
 [0, 0, 0, 5],
 [0, 0, 0, 16],
 [0, 0, 0, 15],
 [0, 0, 0, 3],
 [0, 0, 1, 47],
 [0, 0, 0, 6],
 [0, 0, 0, 6],
 [0, 0, 0, 10],
 [0, 0, 0, 5],
 [0, 0, 0, 2],
 [0, 0, 0, 4],
 [0, 0, 0, 13],
 [0, 0, 0, 6],
 [0, 0, 0, 1],
 [0, 0, 0, 3],
 [0, 0, 0, 3],
 [0, 0, 0, 4],
 [0, 0, 0, 3],
 [0, 0, 0, 2],
 [0, 0, 0, 4],
 [0, 0, 0, 10],
 [0, 0, 0, 1],
 [0, 0, 0, 9],
 [0, 1, 0, 22],
 [0, 0, 0, 5],
 [0, 0, 0, 3],
 [1, 0, 0, 16],
 [0, 0, 0, 23],
 [0, 0, 1, 54],
 [0, 0, 0, 8],
 [0, 0, 0, 2],
 [0, 0, 0, 1],
 [0, 0, 0, 23],
 [0, 0, 0, 6],
 [0, 0, 0, 1],
 [0, 0, 0, 2]]

In [60]:
# p_value < 0.05 이므로 귀무가설 기각
# 흡연구역, 투기지역은 연관 있음
chi2_contingency(rows, correction=False)

(39.583536163768485,
 0.9999999999924872,
 114,
 array([[2.80112045e-03, 2.80112045e-03, 5.60224090e-03, 9.88795518e-01],
        [8.40336134e-03, 8.40336134e-03, 1.68067227e-02, 2.96638655e+00],
        [1.40056022e-02, 1.40056022e-02, 2.80112045e-02, 4.94397759e+00],
        [1.40056022e-02, 1.40056022e-02, 2.80112045e-02, 4.94397759e+00],
        [4.48179272e-02, 4.48179272e-02, 8.96358543e-02, 1.58207283e+01],
        [4.20168067e-02, 4.20168067e-02, 8.40336134e-02, 1.48319328e+01],
        [8.40336134e-03, 8.40336134e-03, 1.68067227e-02, 2.96638655e+00],
        [1.34453782e-01, 1.34453782e-01, 2.68907563e-01, 4.74621849e+01],
        [1.68067227e-02, 1.68067227e-02, 3.36134454e-02, 5.93277311e+00],
        [1.68067227e-02, 1.68067227e-02, 3.36134454e-02, 5.93277311e+00],
        [2.80112045e-02, 2.80112045e-02, 5.60224090e-02, 9.88795518e+00],
        [1.40056022e-02, 1.40056022e-02, 2.80112045e-02, 4.94397759e+00],
        [5.60224090e-03, 5.60224090e-03, 1.12044818e-02, 1.97759

In [61]:
# 0.60 이상이므로 연관이 있음
x2 = chi2_contingency(rows, correction=False)[0]
n = np.sum(rows)
minDimension = min(np.array(rows).shape)-1

V = np.sqrt((x2/n) / minDimension)

print(V)

0.19224832254107846


### 전체 상관 df

In [38]:
# 함수 정의 및 혼동행렬 생성
def cramers_V(var1, var2):
    crosstab = np.array(pd.crosstab(var1, var2, rownames=None, colnames=None))
    stat = chi2_contingency(crosstab)[0]
    obs = np.sum(crosstab)
    phi2 = stat / obs
    r, k = crosstab.shape
    phi2corr = max(0, phi2 - (((k-1)*(r-1))/(obs-1)))
    rcorr = r - ((r-1)**2)/(obs-1)
    kcorr = k - ((k-1)**2)/(obs-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [39]:
rows = []

for var1 in data_encoded:
    col = []
    for var2 in data_encoded:
        cramers = cramers_V(data_encoded[var1], data_encoded[var2])
        col.append(round(cramers, 2))
    rows.append(col)

In [56]:
cramers_results = np.array(rows)
cramers_results

array([[  0,   0,   1,   0],
       [  0,   0,   1,   0],
       [  1,   1,   0,   2],
       [  0,   0,   0,  59],
       [  0,   0,   0, 292]])

In [41]:
df = pd.DataFrame(cramers_results, columns = data_encoded.columns, index = data_encoded.columns)
df

,흡연구역,쓰레기통,투기지역
흡연구역,1.0,0.0,0.0
쓰레기통,0.0,1.0,0.7
투기지역,0.0,0.7,1.0
